In [ ]:
# !pip uninstall -y tensorflow h5py
# !pip install tensorflow==2.1 h5py==2.10

# Setup

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [ ]:
# %cd drive/MyDrive/skripsi/

In [1]:
import tensorflow as tf, matplotlib.pyplot as plt, os, numpy as np, matplotlib.pyplot as plt, matplotlib.patches as patches
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from framework.utils import bbox_utils, data_utils, drawing_utils, eval_utils, io_utils, train_utils
from framework.models import faster_rcnn

tf.__version__  # if not 2.0.0 please uncomment pip uninstall and install, make sure your python version is 3.7.x

'2.1.0'

In [2]:
import h5py
h5py.__version__  # if not 2.10.0 please uncomment pip uninstall and install, make sure your python version is 3.7.x

'2.10.0'

# Read Dataset from TFRecord

In [3]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
IMAGE_SIZE = 512  # make sure had same size with the picture

In [4]:
def read_tfrecord(example):
    tfrecord_format = {
        "filename": tf.io.FixedLenFeature([], tf.string),
        "pic": tf.io.FixedLenFeature([], tf.string),
        "bbox": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    filename = tf.cast(example["filename"], tf.string)
    image = tf.io.parse_tensor(example["pic"], out_type = tf.uint8)
    bbox = tf.io.parse_tensor(example["bbox"], out_type = tf.float32)
    label = tf.io.parse_tensor(example["label"], out_type = tf.int32)
    return {"filename": filename, "image": image, "bbox": bbox, "label": label}

In [5]:
def load_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames)  # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order)  # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_tfrecord)
    return dataset

In [6]:
def get_dataset(filenames):
    dataset = load_dataset(filenames)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    return dataset

In [7]:
def read_label_map(label_map_path):
    item_id = None
    item_name = None
    items = {}

    with open(label_map_path, "r") as file:
        for line in file:
            line.replace(" ", "")
            if line == "item{":
                pass
            elif line == "}":
                pass
            elif "id" in line:
                item_id = int(line.split(":", 1)[1].strip())
            elif "name" in line:
                item_name = line.split(":", 1)[1].replace("'", "").replace("\"", "").strip()

            if item_id is not None and item_name is not None:
                items[item_name] = item_id
                item_id = None
                item_name = None

    return items

In [8]:
label_map_path = "./data_preparation/label_map.pbtxt"
label_map_dict = read_label_map(label_map_path)

In [9]:
def get_label_text(result, doc = label_map_dict):
    for key, value in doc.items():
        if(value == result + 1):
            return key
    return "Unpredictable"

In [10]:
train_data = get_dataset("./data_preparation/train.tfrecord")

test_data = get_dataset("./data_preparation/test.tfrecord")

train_data

<PrefetchDataset shapes: {filename: (), image: <unknown>, bbox: <unknown>, label: <unknown>}, types: {filename: tf.string, image: tf.uint8, bbox: tf.float32, label: tf.int32}>

In [11]:
def show_data(data, n):
    print(data)
    for dat in data.take(n):
        plt.imshow(dat["image"])
        for coord in dat["bbox"]: # bbox is ymin, xmin, ymax, xmax
            coord *= IMAGE_SIZE
            rect = patches.Rectangle(
                (coord[1].numpy(), coord[0].numpy()),  # x1, y1
                coord[3].numpy() - coord[1].numpy(),  # width
                coord[2].numpy() - coord[0].numpy(),  # height
                linewidth = 2, edgecolor = "r", fill = False)
            plt.gca().add_patch(rect)
        plt.show()

In [12]:
# show_data(train_data, 20)

# Train Model

In [13]:
batch_size = 2
epochs = 10

backbone = "vgg16"
# backbone = "mobilenet_v2"
# backbone = "resnet50"

hyper_params = train_utils.get_hyper_params(backbone)
train_total_item = len(list(train_data))

In [14]:
labels = list(label_map_dict.keys())
# We add 1 class for background
hyper_params["total_labels"] = len(labels) + 1
train_data = train_data.map(lambda data : data_utils.preprocessing_before_frcnn(data, IMAGE_SIZE, IMAGE_SIZE))

In [15]:
data_shapes = data_utils.get_data_shapes()
padding_values = data_utils.get_padding_values()
train_data = train_data.padded_batch(batch_size, padded_shapes=data_shapes, padding_values=padding_values)

In [16]:
anchors = bbox_utils.generate_anchors(hyper_params)
frcnn_train_feed = train_utils.faster_rcnn_generator(train_data, anchors, hyper_params)

In [17]:
if (backbone == "vgg16"):
    from framework.models.rpn_vgg16 import get_rpn_model
elif (backbone == "mobilenet_v2"):
    from framework.models.rpn_mobilenet_v2 import get_rpn_model
elif (backbone == "resnet50"):
    from framework.models.rpn_resnet50 import get_rpn_model

rpn_model, feature_extractor = get_rpn_model(hyper_params)
frcnn_model = faster_rcnn.get_model_frcnn(feature_extractor, rpn_model, anchors, hyper_params)
frcnn_model.compile(optimizer=tf.optimizers.SGD(momentum=8e-1),
                    loss=[None] * len(frcnn_model.output))
faster_rcnn.init_model_frcnn(frcnn_model, hyper_params)

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512, 512, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 512, 512, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 512, 512, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 256, 256, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 256, 256, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 256, 256, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, 128, 128)     0     

In [18]:
# Load weights
frcnn_model_path = io_utils.get_model_path("faster_rcnn", backbone)

load_weights = False
if load_weights:
    frcnn_model.load_weights(frcnn_model_path)
log_path = io_utils.get_log_path("faster_rcnn", backbone)

checkpoint_callback = ModelCheckpoint(frcnn_model_path, monitor="loss", save_best_only=True, save_weights_only=True)
# tensorboard_callback = TensorBoard(log_dir=log_path)
schedule_lr_callback = tf.keras.callbacks.LearningRateScheduler(lambda ep: 1e-5 * 10 ** (ep / 30))

In [19]:
step_size_train = train_utils.get_step_size(train_total_item, batch_size)
history = frcnn_model.fit(frcnn_train_feed,
                steps_per_epoch=step_size_train,
                verbose = 1,
                epochs=epochs,
                callbacks=[checkpoint_callback, schedule_lr_callback])

Train for 890 steps
Epoch 1/10
  1/890 [..............................] - ETA: 3:00:30WARNING:tensorflow:Can save best model only with loss available, skipping.


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[2,1024,32,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model_1/rpn_conv/Conv2D (defined at C:\Users\Rudy Rachman\AppData\Local\Temp\ipykernel_15508\869298414.py:6) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[model_1/roi_bboxes/Reshape_2/_34]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[2,1024,32,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model_1/rpn_conv/Conv2D (defined at C:\Users\Rudy Rachman\AppData\Local\Temp\ipykernel_15508\869298414.py:6) ]]
Hint: If you want to see a list of

# Evaluate Model

In [ ]:
labels = ["bg"] + labels
test_total_item = len(list(test_data))

test_data = test_data.map(lambda data : data_utils.preprocessing_before_frcnn(
                              data, IMAGE_SIZE, IMAGE_SIZE))

test_data = test_data.padded_batch(
    batch_size, padded_shapes=data_shapes, padding_values=padding_values)

In [ ]:
load_path = io_utils.get_model_path("faster_rcnn", backbone)
rpn_model, feature_extractor = get_rpn_model(hyper_params)
frcnn_test_model = faster_rcnn.get_model_frcnn(feature_extractor, rpn_model, anchors, hyper_params, mode="test")
frcnn_test_model.load_weights(load_path)

In [ ]:
frcnn_test_model.summary()

In [ ]:
step_size = train_utils.get_step_size(test_total_item, batch_size)
pred_bboxes, pred_labels, pred_scores = frcnn_test_model.predict(test_data, steps=step_size, verbose=1)

In [ ]:
pred_bboxes
for i in pred_bboxes:
  for j in i:
    for result in j:
      if result != 0:
        print("Berhasil")

In [ ]:
pred_labels

In [ ]:
pred_scores

In [ ]:
eval_utils.evaluate_predictions(test_data, pred_bboxes, pred_labels, pred_scores, labels, batch_size)